# TSP Problem

In [218]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from itertools import combinations
import networkx as nx

from icecream import ic

## Useful functions and declarations

In [219]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km


def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Greedy starting solution

In [220]:
segments = [
    ((c1, c2, float(DIST_MATRIX[c1, c2]))) for c1, c2 in combinations(range(len(CITIES)), 2)
]

G = nx.Graph()
G.add_weighted_edges_from(segments)

segments.sort(key=lambda x: x[2], reverse=True)

starting_node = segments[0][0]
tsp = []
visited = np.full(len(CITIES), False)

current_city = starting_node
tsp.append(current_city)
visited[current_city] = True

while not np.all(visited):
    neighbors = sorted(G[current_city], key=lambda x: G[current_city][x]['weight'])
 
    for n in neighbors:
        if not visited[n]:
            next_city = n
            break

    tsp.append(next_city)
    visited[next_city] = True
    current_city = next_city

tsp.append(starting_node)

ic(tsp_cost(tsp))
None

ic| tsp_cost(tsp): np.float64(5164.960063767532)
